In [1]:
import os
import yaml

config = yaml.safe_load(open("myconfig.yml"))

os.environ["OPENAI_API_KEY"] = config["OPEN_AI_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
os.environ["WANDB_PROJECT"] = config["WANDB_PROJECT"]

# Tools

### Memory as a retrieval tool

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

In [6]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

### Langchain tools

More tools here https://python.langchain.com/docs/integrations/tools

In [9]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

In [8]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [11]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
agent_chain = initialize_agent(
    [retriever_tool, tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [12]:
# run the agent
agent_chain.run(
    "What happened in the latest burning man floods?",
)

/Users/oleksandrhonchar/Documents/GitHub/intro_to_llm_agents/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Streaming LangChain activity to W&B at https://wandb.ai/rachnogstyle/nlw_jan24/runs/u4zzrqjs
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.




> Entering new AgentExecutor chain...
Thought: I should use the Tavily search tool to find information about the latest Burning Man floods.
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": "Latest Burning Man floods news"
}
```
Observation: [{'url': 'https://apnews.com/article/burning-man-flooding-nevada-stranded-0726190c9f8378935e2a3cce7f154785', 'content': 'This photo, provided by Maxar Technologies, shows an overview of the center camp at the Burning Man festival on Monday, Sept. 4, 2023, in the Black Rock Desert north of Reno, Nev. Partygoers stranded for days at the counterculture festival by a late summer storm were allowed to start leaving Monday afternoon after muddy roads dried up enough for them to begin their exodus from the northern Nevada desert. This photo, provided by Maxar Technologies, shows an overview of the center camp at the Burning Man festival on Monday, Sept. 4, 2023, in the Black Rock Desert north of Reno, Nev. Partygoers stranded for

'The latest Burning Man festival experienced flooding that stranded partygoers for days at the counterculture festival due to a late summer storm. The festival attendees were allowed to start leaving after muddy roads dried up enough for them to begin their exodus from the northern Nevada desert. Additionally, there was an investigation into a death at the festival site during the flooding.\n\nIs there anything else you would like to know about the Burning Man floods?\n'

### Custom tools